## 결측치 및 이상치 처리 방안(KNN)
1. 사출온도 설정값 = 0  지움(48개)
2. k_rpm 실제값 = 0 & 40 지움 (이상치)
3. 물 & 손 무게 0으로 바꿈.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
## 데이터 셋 DF로 전환
dataDF = pd.read_csv(r'C:\Users\KDP-14\Desktop\VSCode\KDT6\기업 프로젝트\데이터 전처리 2차\clean_data2.csv')

In [4]:
dataDF

,c_temp_pv,k_rpm_pv,n_temp_pv,scale_pv,s_temp_pv
0,69.6,189,67.2,3.01,67.1
1,69.8,189,67.2,3.01,67.0
2,69.7,189,67.9,3.08,65.9
3,69.7,189,67.8,3.08,65.9
4,69.7,189,67.8,3.08,65.9
...,...,...,...,...,...
232022,69.7,191,67.6,0.00,67.3
232023,69.8,191,67.5,0.00,67.0
232024,69.7,191,67.4,0.00,66.8
232025,69.7,191,67.3,0.00,66.7


In [5]:
dataDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232027 entries, 0 to 232026
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   c_temp_pv  232027 non-null  float64
 1   k_rpm_pv   232027 non-null  int64  
 2   n_temp_pv  232027 non-null  float64
 3   scale_pv   232027 non-null  float64
 4   s_temp_pv  232027 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 8.9 MB


In [6]:
## 물, 손 무게 0으로 바꿈
list(dataDF['scale_pv'].value_counts().index.sort_values())
# 어디까지가 물이고 어디까지가 고무고 어디까지가 사람 손이냐
# 2.0 = 1개 -> 버림(개수가 적고 유효한 값이 아니라고 판단)
# 

[0.0,
 0.01,
 0.02,
 0.03,
 0.04,
 0.05,
 0.06,
 0.07,
 0.08,
 0.1,
 0.11,
 0.12,
 0.13,
 0.15,
 0.16,
 0.17,
 0.18,
 0.19,
 0.2,
 0.21,
 0.22,
 0.23,
 0.39,
 0.82,
 0.85,
 2.0,
 2.56,
 2.57,
 2.69,
 2.71,
 2.72,
 2.73,
 2.74,
 2.75,
 2.76,
 2.77,
 2.78,
 2.79,
 2.8,
 2.81,
 2.82,
 2.83,
 2.84,
 2.85,
 2.86,
 2.87,
 2.88,
 2.89,
 2.9,
 2.91,
 2.92,
 2.93,
 2.94,
 2.95,
 2.96,
 2.97,
 2.98,
 2.99,
 3.0,
 3.01,
 3.02,
 3.03,
 3.04,
 3.05,
 3.06,
 3.07,
 3.08,
 3.09,
 3.1,
 3.11,
 3.12,
 3.13,
 3.14,
 3.15,
 3.16,
 3.17,
 3.18,
 3.19,
 3.2,
 3.21,
 3.22,
 3.23,
 3.24,
 3.25,
 3.26,
 3.27,
 3.28,
 3.29,
 3.34,
 3.35,
 3.37,
 3.41,
 3.43,
 3.49,
 9.17,
 9.18,
 15.1,
 15.3,
 655.04,
 655.19,
 655.2,
 655.21,
 655.22,
 655.23,
 655.24,
 655.25,
 655.26,
 655.27,
 655.28,
 655.29,
 655.3,
 655.31,
 655.32,
 655.33,
 655.34,
 655.35]

In [7]:
## 1. 넓은 범위로 생각해보기(2.7~3.3) 2.9 ~ 3.1까지 정상 데이터로 처리

def change_scale(dataDF, x, y):
    # scale_pv가 x와 y 사이인 값은 유지, 그렇지 않은 값은 0으로 설정
    dataDF['scale_pv'] = dataDF['scale_pv'].apply(lambda value: value if float(x) <= value <= float(y) else 0)
    return dataDF


In [8]:
change_scale(dataDF,2.7,3.3)

,c_temp_pv,k_rpm_pv,n_temp_pv,scale_pv,s_temp_pv
0,69.6,189,67.2,3.01,67.1
1,69.8,189,67.2,3.01,67.0
2,69.7,189,67.9,3.08,65.9
3,69.7,189,67.8,3.08,65.9
4,69.7,189,67.8,3.08,65.9
...,...,...,...,...,...
232022,69.7,191,67.6,0.00,67.3
232023,69.8,191,67.5,0.00,67.0
232024,69.7,191,67.4,0.00,66.8
232025,69.7,191,67.3,0.00,66.7


In [9]:
dataDF['scale_pv'].value_counts().index.sort_values()

Index([ 0.0, 2.71, 2.72, 2.73, 2.74, 2.75, 2.76, 2.77, 2.78, 2.79,  2.8, 2.81,
       2.82, 2.83, 2.84, 2.85, 2.86, 2.87, 2.88, 2.89,  2.9, 2.91, 2.92, 2.93,
       2.94, 2.95, 2.96, 2.97, 2.98, 2.99,  3.0, 3.01, 3.02, 3.03, 3.04, 3.05,
       3.06, 3.07, 3.08, 3.09,  3.1, 3.11, 3.12, 3.13, 3.14, 3.15, 3.16, 3.17,
       3.18, 3.19,  3.2, 3.21, 3.22, 3.23, 3.24, 3.25, 3.26, 3.27, 3.28, 3.29],
      dtype='float64', name='scale_pv')

In [10]:
dataDF.replace(0,np.nan,inplace=True)

In [11]:
dataDF['scale_pv'].value_counts().index.sort_values()

Index([2.71, 2.72, 2.73, 2.74, 2.75, 2.76, 2.77, 2.78, 2.79,  2.8, 2.81, 2.82,
       2.83, 2.84, 2.85, 2.86, 2.87, 2.88, 2.89,  2.9, 2.91, 2.92, 2.93, 2.94,
       2.95, 2.96, 2.97, 2.98, 2.99,  3.0, 3.01, 3.02, 3.03, 3.04, 3.05, 3.06,
       3.07, 3.08, 3.09,  3.1, 3.11, 3.12, 3.13, 3.14, 3.15, 3.16, 3.17, 3.18,
       3.19,  3.2, 3.21, 3.22, 3.23, 3.24, 3.25, 3.26, 3.27, 3.28, 3.29],
      dtype='float64', name='scale_pv')

## KNN을 이용해서 결측치 채우기

In [12]:
# 모듈 로딩
from sklearn.impute import KNNImputer

In [13]:
imputer = KNNImputer(
                     n_neighbors=7,
                     weights='uniform'
                     )

In [14]:
dataDF

,c_temp_pv,k_rpm_pv,n_temp_pv,scale_pv,s_temp_pv
0,69.6,189,67.2,3.01,67.1
1,69.8,189,67.2,3.01,67.0
2,69.7,189,67.9,3.08,65.9
3,69.7,189,67.8,3.08,65.9
4,69.7,189,67.8,3.08,65.9
...,...,...,...,...,...
232022,69.7,191,67.6,NaN,67.3
232023,69.8,191,67.5,NaN,67.0
232024,69.7,191,67.4,NaN,66.8
232025,69.7,191,67.3,NaN,66.7


In [15]:
dataDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232027 entries, 0 to 232026
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   c_temp_pv  232027 non-null  float64
 1   k_rpm_pv   232027 non-null  int64  
 2   n_temp_pv  232027 non-null  float64
 3   scale_pv   38609 non-null   float64
 4   s_temp_pv  232027 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 8.9 MB


In [16]:
imputed_data = imputer.fit_transform(dataDF)

In [17]:
imputed_df = pd.DataFrame(imputed_data, columns=dataDF.columns)

In [18]:
imputed_df['scale_pv']

0         3.010000
1         3.010000
2         3.080000
3         3.080000
4         3.080000
            ...   
232022    3.067143
232023    3.065714
232024    3.061429
232025    3.058571
232026    3.068571
Name: scale_pv, Length: 232027, dtype: float64

In [19]:
imputed_df.to_csv('KNN 전처리2(2.7~3.3_n7).csv',encoding='utf-8',index=False)